In [8]:
!pip install -r ../requirements.txt

In [9]:
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import pandas as pd
from bs4 import BeautifulSoup
import csv
import sys
# This is the website which we will be scraping
base_url = "https://www.cvedetails.com/"
s = requests.Session()

retries = Retry(total=100,
                backoff_factor=0.5,
                status_forcelist= [500, 502, 503, 504])

s.mount('https://', HTTPAdapter(max_retries=retries))

In [10]:
# headers = ["#", "CVE ID", "CWE ID", "# of Exploits", "Vulnerability Type(s)", "Publish Date", "Update Date", "Score", "Gained Access Level", "Access", "Complexity", "Authentication", "Confidentiality", "Integrity", "Availability",  "Vendor", "Product"]
headers = ["#", "CVE ID", "CWE ID", "# of Exploits", "Vulnerability Type(s)", "Publish Date", "Update Date", "Score", "Gained Access Level", "Access", "Complexity", "Authentication", "Confidentiality", "Integrity", "Availability", "Desc", "Vendor", "Product"]
with open("vul_data_desc.csv", "w") as f:
    write = csv.writer(f)
    write.writerow(headers)

def write_row(data):
    with open("vul_data_desc.csv", "a") as f:
        writer = csv.writer(f)
        writer.writerow(data)
        

In [11]:
def scrape_table_vul(url):
    search_vul_page = BeautifulSoup(s.get(url).content, "html.parser")
    # print(url)
    for row, desc in zip(search_vul_page.find_all(class_="srrowns"), search_vul_page.find_all(class_="cvesummarylong")):
        # print(list(zip(headers, list(map(lambda x: x.get_text().strip(), row.find_all("td"))))))
        
        # Get the row of the table
        csv_row = list(map(lambda x: x.get_text().strip(), row.find_all("td")))

        # Add the description to the row
        csv_row.append(desc.get_text().strip())

        # Get the Product and Vendor info and add it to the row
        cve_url = base_url + "/cve/" + csv_row[1]
        cve_page = BeautifulSoup(s.get(cve_url).content, "html.parser")
        if cve_page and cve_page.find(id="vulnversconuttable"):
            for x in cve_page.find(id="vulnversconuttable").find_all("td")[:2]:
                csv_row.append(x.get_text().strip())
        else:
            print(cve_url)
        
        
        write_row(csv_row)
        

In [12]:
def scrape_year_vul(url):
    year_vul_page = BeautifulSoup(s.get(url).content, "html.parser")
    pages = []
    for page_div in year_vul_page.find_all(class_="paging"):
        for page in page_div.find_all("a"):
            pages.append(base_url + page["href"])
    
    # print(pages)
    for page in pages:
        scrape_table_vul(page)
        # print(page)
    
    

In [13]:
def scrape_vul():
    
   
    # Get the urls for each of the years
    url = "https://www.cvedetails.com/browse-by-date.php"
    vul_page = BeautifulSoup(s.get(url).content, "html.parser")
    year_urls = []
    for i, year in enumerate(vul_page.find(class_="stats").find_all("th")):
        if year.find("a"):
            year_urls.append(base_url + year.find("a")["href"])
    # print(year_urls)
    
    # Go through each of the pages and add vulenerabilites to the table
    for year_url in year_urls:
        print(year_url)
        scrape_year_vul(year_url)

scrape_vul()

https://www.cvedetails.com//vulnerability-list/year-1999/vulnerabilities.html
https://www.cvedetails.com//cve/CVE-1999-1592
https://www.cvedetails.com//cve/CVE-1999-0944
https://www.cvedetails.com//cve/CVE-1999-0935
https://www.cvedetails.com//cve/CVE-1999-0934
https://www.cvedetails.com//cve/CVE-1999-0698
https://www.cvedetails.com//cve/CVE-1999-0678
https://www.cvedetails.com//cve/CVE-1999-0665
https://www.cvedetails.com//cve/CVE-1999-0664
https://www.cvedetails.com//cve/CVE-1999-0663
https://www.cvedetails.com//cve/CVE-1999-0662
https://www.cvedetails.com//cve/CVE-1999-0661
https://www.cvedetails.com//cve/CVE-1999-0657
https://www.cvedetails.com//cve/CVE-1999-0654
https://www.cvedetails.com//cve/CVE-1999-0653
https://www.cvedetails.com//cve/CVE-1999-0651
https://www.cvedetails.com//cve/CVE-1999-0650
https://www.cvedetails.com//cve/CVE-1999-0641
https://www.cvedetails.com//cve/CVE-1999-0640
https://www.cvedetails.com//cve/CVE-1999-0639
https://www.cvedetails.com//cve/CVE-1999-0638
ht

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://www.cvedetails.com//cve/CVE-2021-25927
https://www.cvedetails.com//cve/CVE-2021-25910
https://www.cvedetails.com//cve/CVE-2021-25909
https://www.cvedetails.com//cve/CVE-2021-25898
https://www.cvedetails.com//cve/CVE-2021-25849
https://www.cvedetails.com//cve/CVE-2021-25848
https://www.cvedetails.com//cve/CVE-2021-25847
https://www.cvedetails.com//cve/CVE-2021-25846
https://www.cvedetails.com//cve/CVE-2021-25845
https://www.cvedetails.com//cve/CVE-2021-25839
https://www.cvedetails.com//cve/CVE-2021-25812
https://www.cvedetails.com//cve/CVE-2021-25811
https://www.cvedetails.com//cve/CVE-2021-25810
https://www.cvedetails.com//cve/CVE-2021-25681
https://www.cvedetails.com//cve/CVE-2021-25680
https://www.cvedetails.com//cve/CVE-2021-25679
https://www.cvedetails.com//cve/CVE-2021-25676
https://www.cvedetails.com//cve/CVE-2021-25671
https://www.cvedetails.com//cve/CVE-2021-25669
https://www.cvedetails.com//cve/CVE-2021-25668
https://www.cvedetails.com//cve/CVE-2021-25667
https://www.c

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://www.cvedetails.com//cve/CVE-2021-22882
https://www.cvedetails.com//cve/CVE-2021-22793
https://www.cvedetails.com//cve/CVE-2021-22774
https://www.cvedetails.com//cve/CVE-2021-22773
https://www.cvedetails.com//cve/CVE-2021-22772
https://www.cvedetails.com//cve/CVE-2021-22771
https://www.cvedetails.com//cve/CVE-2021-22770
https://www.cvedetails.com//cve/CVE-2021-22769
https://www.cvedetails.com//cve/CVE-2021-22768
https://www.cvedetails.com//cve/CVE-2021-22767
https://www.cvedetails.com//cve/CVE-2021-22766
https://www.cvedetails.com//cve/CVE-2021-22765
https://www.cvedetails.com//cve/CVE-2021-22764
https://www.cvedetails.com//cve/CVE-2021-22763
https://www.cvedetails.com//cve/CVE-2021-22749
https://www.cvedetails.com//cve/CVE-2021-22747
https://www.cvedetails.com//cve/CVE-2021-22746
https://www.cvedetails.com//cve/CVE-2021-22745
https://www.cvedetails.com//cve/CVE-2021-22744
https://www.cvedetails.com//cve/CVE-2021-22743
https://www.cvedetails.com//cve/CVE-2021-22742
https://www.c

In [14]:
vul = pd.read_csv("vul_data.csv")
print(vul.shape[0])

162250
